In [1]:
#import all keys and packages for tweepy and pandas
import datetime
import tweepy 
import os
import pandas as pd

from API_keys import api_key, api_key_secret, access_token, access_token_secret

In [2]:
# Authenticate the Tweepy API
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [3]:
date_since = "2020-10-14" #went a more manual way to get the date rather than datetime, but either works
search_words = "#hunting" + '-filter:retweets'

In [14]:
#created a variable to store the api search rather for future calls to pull data below and change up the output.

tweets = tweepy.Cursor(api.search,
               # tweet_mode is defaulted to short, which only holds the first 140 characters of a Tweet.
               tweet_mode='extended',
               q=search_words,
               lang='en', since = date_since).items()



       

In [15]:
#list comprehension to pull date, user, location, and tweet

user_info = [[item.created_at,item.user.screen_name, item.user.location, item.full_text] for item in tweets]


In [17]:
#placed the user info into a dataframe for future analyzing within SQL or python

tweet_text = pd.DataFrame(data=user_info, columns=['date', 'user', 'location', 'tweet'])



In [18]:
tweet_text

,date,user,location,tweet
0,2020-10-22 14:41:43,CaloosaSales,"Fort Myers, Florida",Shoutout to one of our sponsors - Cablz! Trust...
1,2020-10-22 14:38:20,GFTrib_KPuckett,Montana,Here we go again. #MTwx #dejavu #hunting http...
2,2020-10-22 14:37:54,NDANickP,"Indiana, PA",Wonderful testimonial about our #fieldtofork p...
3,2020-10-22 14:36:02,Win_Win_Now,,"Please Retweet.\n\nWin a $4,000 #Camping Prize..."
4,2020-10-22 14:27:27,Inle_rahbit,UK,@nationaltrust @JudeAvalon You post this answe...
...,...,...,...,...
1705,2020-10-14 08:20:41,Big_G_09,"Saskatoon, Saskatchewan 🇨🇦",@ToddAndreas @DwayneMihalicz #startemyoung 👍 #...
1706,2020-10-14 07:37:20,at_ease_shop,,One of our favorite pieces: The @ColumbiaSA #G...
1707,2020-10-14 07:25:01,TacticalSHT,"St Louis, MO","""tacshit2020"" SAVES 15% OFF\n👇👇\nCheck it Out!..."
1708,2020-10-14 07:24:56,frugalmaniac,"MN, USA",It's a #fishing #Hunting #camping anything out...


In [19]:
tweet_text.to_csv('hunting_tweets.csv')